In [9]:
import pandas as pd
import numpy as np
import os
import requests
import env
import john_acquire as a

In [10]:
%load_ext autoreload
%autoreload 2

# Set the option to display all columns
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Load the acquired data
inspection_df = pd.read_csv('nyc_health_inspections_2000_to_2023.csv', index_col=False)
inspection_df.head()

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description,grade,grade_date
0,50067297,GERBASI RESTAURANT,Bronx,2389,ARTHUR AVENUE,10458.0,7182205735,Italian,2021-09-12T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:08.000,Inter-Agency Task Force / Initial Inspection,40.855290,-73.887797,206.0,15.0,39100.0,2011897.0,2.030650e+09,BX06,NaN,NaN,NaN,NaN
1,50034232,RELISH CATERERS,Bronx,2501,3 AVENUE,10451.0,2122281672,American,2021-09-25T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:08.000,Inter-Agency Task Force / Initial Inspection,40.810202,-73.928401,201.0,8.0,5100.0,2000795.0,2.023200e+09,BX39,NaN,NaN,NaN,NaN
2,50064240,DAXI SICHUAN,Queens,136-20,ROOSEVELT AVENUE,11354.0,9175631983,Chinese,2022-09-21T00:00:00.000,Violations were cited in the following area(s).,Not Critical,13.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.759778,-73.829235,407.0,20.0,85300.0,4113546.0,4.050190e+09,QN22,09B,Thawing procedure improper.,A,2022-09-21T00:00:00.000
3,50105603,LE PAIN QUOTIDIEN,Manhattan,81,WEST BROADWAY,10007.0,6468639168,French,2022-11-25T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,NaN,2023-12-01T06:00:09.000,Administrative Miscellaneous / Re-inspection,40.715083,-74.009567,101.0,1.0,2100.0,1001480.0,1.001368e+09,MN24,NaN,NaN,NaN,NaN
4,50069583,PHO BEST,Queens,4235,MAIN ST,11355.0,9173618878,Southeast Asian,2022-05-09T00:00:00.000,Violations were cited in the following area(s).,Critical,30.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.754418,-73.827881,407.0,20.0,85300.0,4573539.0,4.051358e+09,QN22,02B,Hot food item not held at or above 140º F.,NaN,NaN


In [12]:
# Find what fields are missing values
inspection_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207365 entries, 0 to 207364
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   camis                  207365 non-null  int64  
 1   dba                    207365 non-null  object 
 2   boro                   207365 non-null  object 
 3   building               207076 non-null  object 
 4   street                 207365 non-null  object 
 5   zipcode                204723 non-null  float64
 6   phone                  207359 non-null  object 
 7   cuisine_description    207365 non-null  object 
 8   inspection_date        207365 non-null  object 
 9   action                 207365 non-null  object 
 10  critical_flag          207365 non-null  object 
 11  score                  199793 non-null  float64
 12  record_date            207365 non-null  object 
 13  inspection_type        207365 non-null  object 
 14  latitude               207119 non-nu

We are should attempt to infer some of the nulls if we can from other information from the row. 
for example, if we knew the bin (building identification number), we could deduce the zip code.

the order is like this
lat&long, building, bin*, bbl*, nta, zipcode*, community board, council district, census track

zipcode can only accuratly be infer using bin or bbl

notice how bbl has very few nulls, we can use this to guess "nta, zipcode*, community board, council district, census track"


In [13]:
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

building                    289
zipcode                    2642
phone                         6
score                      7572
latitude                    246
longitude                   246
community_board            3153
council_district           3157
census_tract               3157
bin                        4144
bbl                         511
nta                        3153
violation_code             1157
violation_description      1157
grade                    104404
grade_date               113157
dtype: int64

However after looking at the values, it contains numbers 1-5 which are not the standard 10 digit building block lot number. 

In [14]:
sorted(inspection_df.bbl.unique())[:10]

[1.0,
 2.0,
 3.0,
 4.0,
 1000000000.0,
 1000010010.0,
 1000020002.0,
 1000030001.0,
 1000047501.0,
 1000057501.0]

In [15]:
bbl_values = [np.nan, 1.0, 2.0, 3.0, 4.0, 5.0]
inspection_df['bbl'].isin(bbl_values).sum()

4144

if we add the total non correct numbers (np.nan, 1.0, 2.0, 3.0, 4.0, 5.0), we total 4144, the identical number of nans in bin column. this leads me to believe that the nans are consistent in common across the identifying columns. 

census_tract               3157
bin                        4144
bbl                        4144
nta                        3153
community_board            3153
council_district           3157

bbl was going to allow us to infer many columns, we could move to the next id level (nta) but there is very small difference in the number of nulls, not to mention the nta is about the same area size as a zip code. Lets drop all the nans in the bin column and count the nans in the dataframe again.  

In [16]:
inspection_df = inspection_df.dropna(subset=['bin'])
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

zipcode                      30
phone                         6
score                      7440
community_board              30
council_district             34
census_tract                 34
nta                          30
violation_code             1076
violation_description      1076
grade                    102689
grade_date               111279
dtype: int64

as expected, the nans were amost all related/in common. 
there are about a 30 nulls left, lets just drop them, too small of a figure. 

In [17]:
inspection_df = inspection_df.dropna(subset=['council_district'])
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

phone                         6
score                      7438
violation_code             1076
violation_description      1076
grade                    102677
grade_date               111265
dtype: int64

only numbers are left, we could simply fill na with 0000000000

In [18]:
inspection_df['phone'].fillna('0000000000', inplace=True)
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

score                      7438
violation_code             1076
violation_description      1076
grade                    102677
grade_date               111265
dtype: int64

all location based nulls are gone!

Next lets look at grade and grade date
because not all inspections receive a grade depending on the inspecton type, we will be dropping these 2 features. we can calculate the grade with the score. The documentation also mentions that the grade may not match the score because of input errors, so its safer this way. 

There is still other fields that have missing values: score, violation_code, violation_description. However, we may be able to handle it by grouping by camis + indpection type + inspections date. If any matching observations have the missing values, we can use that data. 

In [19]:
inspection_df = inspection_df.drop(['grade', 'grade_date'], axis=1)
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

score                    7438
violation_code           1076
violation_description    1076
dtype: int64

inspection outcome related nulls Lets first find out what type of inspections there are, we are only focus on inspections related to food safety for now. 

In [20]:
# Assuming 'inspection_df' is your DataFrame
unique_inspection_types = inspection_df['inspection_type'].unique()

# Convert the numpy array to a list and then sort it
sorted_inspection_types = sorted(unique_inspection_types.tolist())
sorted_inspection_types

['Administrative Miscellaneous / Compliance Inspection',
 'Administrative Miscellaneous / Initial Inspection',
 'Administrative Miscellaneous / Re-inspection',
 'Administrative Miscellaneous / Reopening Inspection',
 'Administrative Miscellaneous / Second Compliance Inspection',
 'Calorie Posting / Compliance Inspection',
 'Calorie Posting / Initial Inspection',
 'Calorie Posting / Re-inspection',
 'Cycle Inspection / Compliance Inspection',
 'Cycle Inspection / Initial Inspection',
 'Cycle Inspection / Re-inspection',
 'Cycle Inspection / Reopening Inspection',
 'Cycle Inspection / Second Compliance Inspection',
 'Inter-Agency Task Force / Initial Inspection',
 'Inter-Agency Task Force / Re-inspection',
 'Pre-permit (Non-operational) / Compliance Inspection',
 'Pre-permit (Non-operational) / Initial Inspection',
 'Pre-permit (Non-operational) / Re-inspection',
 'Pre-permit (Non-operational) / Second Compliance Inspection',
 'Pre-permit (Operational) / Compliance Inspection',
 'Pre-per

Inspection types : "Calorie Posting", "Pre-permit", "Smoke-Free Air Act", "Trans Fat" seem to be a diferent type of inspection that does not directly pertain to food safety, we can drop theses so we can focus on the other types: Administrative, Cycle and Inter-Agency 

In [21]:
# List of inspection types to be removed
remove_types = ["Calorie Posting", "Pre-permit", "Smoke-Free Air Act", "Trans Fat"]

# Filter the DataFrame in a single step
inspection_df = inspection_df[~inspection_df['inspection_type'].str.startswith(tuple(remove_types))]
len(inspection_df)

155970

In [22]:
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

score                    6024
violation_code            797
violation_description     797
dtype: int64

there were some rows in common that had nulls, so dorpping them we saw a small decrease in nulls overall. we still have a few violation codes left to look into

Lets see the history of a restaurant with a null value in the violation code and try to figure out why. 

In [24]:
# Step 1: Group by 'camis' and 'inspection_date' and check for nulls in 'violation_code'
grouped = inspection_df.groupby(['camis', 'inspection_date'])
groups_with_nulls = grouped.apply(lambda x: x['score'].isna().any())

# Step 2: Filter the DataFrame to include only those groups
filtered_df = inspection_df[inspection_df.set_index(['camis', 'inspection_date']).index.isin(groups_with_nulls[groups_with_nulls].index)].reset_index(drop=True)

# Now, 'filtered_df' contains only the groups where there are null values in 'violation_code'
filtered_df.sort_values(by='camis').head(3)


,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description
5179,30112340,WENDY'S,Brooklyn,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2022-07-13T00:00:00.000,Violations were cited in the following area(s).,Critical,11.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.662652,-73.962081,309.0,40.0,32700.0,3029737.0,3.011970e+09,BK60,02G,Cold TCS food item held above 41 °F; smoked or...
2213,30112340,WENDY'S,Brooklyn,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2022-07-13T00:00:00.000,Violations were cited in the following area(s).,Not Critical,NaN,2023-12-01T06:00:08.000,Administrative Miscellaneous / Initial Inspection,40.662652,-73.962081,309.0,40.0,32700.0,3029737.0,3.011970e+09,BK60,20-06,Current letter grade or Grade Pending card not...
3508,30112340,WENDY'S,Brooklyn,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2022-07-13T00:00:00.000,Violations were cited in the following area(s).,Not Critical,11.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.662652,-73.962081,309.0,40.0,32700.0,3029737.0,3.011970e+09,BK60,10F,Non-food contact surface or equipment made of ...


using the date, we can see that in a single inspection, each violation is in a new row. from here we can see that a single inspection can have more thna one inspection type. Administrative Miscellaneous seem to be the ones that hold the NaN data. We should figure out how many Administrative types have nan

In [25]:
# Group by 'inspection_type' and count null 'violation_code' entries
null_score_count = inspection_df.groupby('inspection_type').apply(lambda x: x['score'].isnull().sum())

# The result is a Series where the index is 'inspection_type' and the values are the counts of null 'violation_code'
print(null_score_count)


inspection_type
Administrative Miscellaneous / Compliance Inspection             99
Administrative Miscellaneous / Initial Inspection              4899
Administrative Miscellaneous / Re-inspection                    975
Administrative Miscellaneous / Reopening Inspection              43
Administrative Miscellaneous / Second Compliance Inspection       8
Cycle Inspection / Compliance Inspection                          0
Cycle Inspection / Initial Inspection                             0
Cycle Inspection / Re-inspection                                  0
Cycle Inspection / Reopening Inspection                           0
Cycle Inspection / Second Compliance Inspection                   0
Inter-Agency Task Force / Initial Inspection                      0
Inter-Agency Task Force / Re-inspection                           0
dtype: int64


We can see that all of the null values come from inspection type Administrative. It could be that this type of row is used to record violations of a different type since there is a mix of admin and cycle inspectons for a single visit. We will probably be able to infer the score or drop the inspection type administrative. I think is is a strong indication that we should probably drop any Administrative inspection as it does not seem to affect the score. Lets take a look at what type of violations we see in this type of inspection. 

In [26]:
# Filter for rows where 'inspection_type' starts with "Administrative"
administrative_rows = inspection_df[inspection_df['inspection_type'].str.startswith("Administrative")]

# Get a count of each unique 'violation_description' in these rows
violation_description_counts = administrative_rows['violation_description'].value_counts()

# Display the counts
violation_description_counts

violation_description
“Choking first aid” poster not posted. “Alcohol and pregnancy” warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted.                                                                                                                                                 763
Food allergy information poster not conspicuously posted where food is being prepared or processed by food workers.                                                                                                                                                                                                                                  699
Current letter grade or Grade Pending card not posted                                                                                                                                                                                                                                           

this does confirm that we should drop the administrative inspections as these rows do not contain food safety violations which we are after. 

In [27]:
# Drop rows where 'inspection_type' starts with "Administrative"
inspection_df = inspection_df[~inspection_df['inspection_type'].str.startswith("Administrative")]
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

violation_code           438
violation_description    438
dtype: int64

As expected, the administravive vinspection types held all of the score nulls, so we have no nans left for score. our violation code/description went down as well. Lets continue to look into these nulls, there are very few left and we can probably drop them but im curios to see why they are there, this info might come in handly later. 

In [28]:
# Group by 'inspection_type' and count null 'violation_code' entries
null_violation_count = inspection_df.groupby('inspection_type').apply(lambda x: x['violation_code'].isnull().sum())
null_violation_count

inspection_type
Cycle Inspection / Compliance Inspection             2
Cycle Inspection / Initial Inspection              256
Cycle Inspection / Re-inspection                    40
Cycle Inspection / Reopening Inspection             70
Cycle Inspection / Second Compliance Inspection      0
Inter-Agency Task Force / Initial Inspection        69
Inter-Agency Task Force / Re-inspection              1
dtype: int64

missing violation code varies by inspeciton type. This does not tell us why, but it was worth looking at the ispection type for any clues. 

Since we want to know why violation code/description is null, we can look at the "action" column to find any hints

In [29]:
violation_code_null = inspection_df[inspection_df['violation_code'].isna()]
# Group by 'inspection_type' and count null 'violation_code' entries
null_violation_count = violation_code_null.groupby('action').apply(lambda x: x['violation_code'].isnull().sum())
null_violation_count

action
Establishment re-opened by DOHMH.                               70
No violations were recorded at the time of this inspection.    364
Violations were cited in the following area(s).                  4
dtype: int64

In [251]:
null_violation_count.sum()

438

this does tell us why there are nulls. It seems that the majority nulls are because no violations were found (424). 

we should look into the few remaining rows with nulls. lets start with the reopening rows. 

In [252]:
# Drop rows where 'inspection_type' starts with "Administrative"
action_reopened = inspection_df[inspection_df['action'].str.startswith("Establishment re-opened by DOHMH")]
len(action_reopened)

1360

In [194]:
action_reopened.head(5)

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description
9,50078860,A FEI CHINESE RESTAURANT,Brooklyn,553,THROOP AVENUE,11216.0,7184535205,Chinese,2022-02-23T00:00:00.000,Establishment re-opened by DOHMH.,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.683160,-73.940966,303.0,36.0,27500.0,3052864.0,3.018410e+09,BK35,NaN,NaN
88,50032753,AGAVI ORGANIC JUICEBAR,Manhattan,72,EAST 7 STREET,10003.0,2123908042,"Juice, Smoothies, Fruit Salads",2022-03-11T00:00:00.000,Establishment re-opened by DOHMH.,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.727390,-73.986766,103.0,2.0,3800.0,1006277.0,1.004480e+09,MN22,NaN,NaN
110,50114990,3 TIMES,Manhattan,9096,CLINTON ST,NaN,9293077777,Chinese,2023-04-07T00:00:00.000,Establishment re-opened by DOHMH.,Critical,13.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,0.000000,0.000000,NaN,NaN,NaN,NaN,1.000000e+00,NaN,04L,Evidence of mice or live mice in establishment...
139,50005590,FAMOUS SICHUAN,Manhattan,10,PELL STREET,10013.0,2122333888,Chinese,2022-09-29T00:00:00.000,Establishment re-opened by DOHMH.,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.714729,-73.997598,103.0,1.0,2900.0,1001776.0,1.001630e+09,MN27,NaN,NaN
227,41549281,DUNKIN,Manhattan,316,WEST 34 STREET,10001.0,2127602600,Donuts,2022-07-29T00:00:00.000,Establishment re-opened by DOHMH.,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.752494,-73.994221,104.0,3.0,10300.0,1013552.0,1.007570e+09,MN13,NaN,NaN


Some reopening inspections have violation code and description, while others have nan. We can assume the ones with nan are because there were no violations found like in the mayority of the nans in this column we discovered earlier. we will be filling these with an appropriate label. for now lets move on to the next.

that leaves us with:
Violations were cited in the following area(s).  5


In [30]:
# Drop rows where 'inspection_type' starts with "Administrative"
action_violationcited = inspection_df[inspection_df['action'].str.startswith("Violations were cited in the following area(s)")]
action_violationcited.head()

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description
2,50064240,DAXI SICHUAN,Queens,136-20,ROOSEVELT AVENUE,11354.0,9175631983,Chinese,2022-09-21T00:00:00.000,Violations were cited in the following area(s).,Not Critical,13.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.759778,-73.829235,407.0,20.0,85300.0,4113546.0,4.050190e+09,QN22,09B,Thawing procedure improper.
4,50069583,PHO BEST,Queens,4235,MAIN ST,11355.0,9173618878,Southeast Asian,2022-05-09T00:00:00.000,Violations were cited in the following area(s).,Critical,30.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.754418,-73.827881,407.0,20.0,85300.0,4573539.0,4.051358e+09,QN22,02B,Hot food item not held at or above 140º F.
7,50089970,FOO ON RESTAURANT,Queens,18304,HILLSIDE AVE,11432.0,7182971287,Chinese,2022-04-08T00:00:00.000,Violations were cited in the following area(s).,Not Critical,19.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.713870,-73.778712,412.0,23.0,47000.0,4212729.0,4.099300e+09,QN61,10J,Hand wash sign not posted
8,40986189,LIEBMAN'S DELI,Bronx,552,WEST 235 STREET,10463.0,7185484534,Jewish/Kosher,2023-10-18T00:00:00.000,Violations were cited in the following area(s).,Critical,23.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.885579,-73.909622,208.0,11.0,29500.0,2084091.0,2.057860e+09,BX29,02B,Hot TCS food item not held at or above 140 °F.
11,41433469,THE CAPITAL GRILLE,Manhattan,120,BROADWAY,10271.0,2123741811,American,2019-10-21T00:00:00.000,Violations were cited in the following area(s).,Critical,11.0,2023-12-01T06:00:08.000,Cycle Inspection / Re-inspection,40.708539,-74.011041,101.0,1.0,700.0,1001026.0,1.000478e+09,MN25,02B,Hot food item not held at or above 140º F.


These shows inspections with action "Violations were cited in the following area(s)." with a mix of nulls and codes, so we can assume that the 5 nulls were in error. to summarize, we will fill the nans according to:

"Establishment re-opened by DOHMH." to No violations were recorded

"No violations were recorded at the time of this inspection." to  No violations were recorded

"Violations were cited in the following area(s)." we will drop these because they dont contain any usefull info

In [32]:
# Identify rows where 'action' starts with the specified strings and 'violation_code' is null
condition = inspection_df['violation_code'].isna() & inspection_df['action'].str.startswith(("Establishment re-opened by DOHMH", "No violations were recorded at the time of this inspection."))

# Update 'violation_code' and 'violation_description' for these rows
inspection_df.loc[condition, ['violation_code', 'violation_description']] = ['none', 'No violations were recorded']

In [34]:
violation_code_null = inspection_df[inspection_df['violation_code'].isna()]
# Group by 'inspection_type' and count null 'violation_code' entries
null_violation_count = violation_code_null.groupby('action').apply(lambda x: x['violation_code'].isnull().sum())
null_violation_count


action
Violations were cited in the following area(s).    4
dtype: int64

In [35]:
inspection_df = inspection_df.drop(inspection_df[(inspection_df['violation_code'].isna()) & (inspection_df['action'].str.startswith("Violations were cited in the following area(s)"))].index)

In [36]:
violation_code_null = inspection_df[inspection_df['violation_code'].isna()]
# Group by 'inspection_type' and count null 'violation_code' entries
null_violation_count = violation_code_null.groupby('action').apply(lambda x: x['violation_code'].isnull().sum())
null_violation_count


,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description
action,,,,,,,,,,,,,,,,,,,,,,,,


In [37]:

null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

Series([], dtype: int64)

that took care of al ll the nulls in the dataframebb

In [38]:
inspection_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149942 entries, 0 to 207364
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   camis                  149942 non-null  int64  
 1   dba                    149942 non-null  object 
 2   boro                   149942 non-null  object 
 3   building               149942 non-null  object 
 4   street                 149942 non-null  object 
 5   zipcode                149942 non-null  float64
 6   phone                  149942 non-null  object 
 7   cuisine_description    149942 non-null  object 
 8   inspection_date        149942 non-null  object 
 9   action                 149942 non-null  object 
 10  critical_flag          149942 non-null  object 
 11  score                  149942 non-null  float64
 12  record_date            149942 non-null  object 
 13  inspection_type        149942 non-null  object 
 14  latitude               149942 non-null  f

In [26]:
# Create an empty DataFrame for comparison
comparison_df = pd.DataFrame(columns=['camis', 'dba', 'boro', 'street', 'bbl', 'original_zipcode', 'new_zipcode'])

# Populate the comparison DataFrame with relevant columns from the original DataFrame 'inspection_df'
comparison_df['camis'] = inspection_df['camis']
comparison_df['dba'] = inspection_df['dba']
comparison_df['boro'] = inspection_df['boro']
comparison_df['street'] = inspection_df['street']
comparison_df['bbl'] = inspection_df['bbl']
comparison_df['original_zipcode'] = inspection_df['zipcode']

# Create a copy of the original DataFrame
temp_df = inspection_df.copy()

# Group by 'street' and 'bbl' and fill missing 'zipcode' values
temp_df['zipcode'] = temp_df.groupby(['street', 'bbl'])['zipcode'].transform(lambda x: x if temp_df['bbl'].isna().all() else x.fillna(x.mode()[0] if not x.mode().empty else x))




# Add the updated 'zipcode' values to the comparison table
comparison_df['new_zipcode'] = temp_df['zipcode']

# Filter out rows where the original and new zipcodes are the same (no change)
unchanged_rows = comparison_df['original_zipcode'] == comparison_df['new_zipcode']
changed_comparison_df = comparison_df[~unchanged_rows]

# Display the DataFrame with changed zipcodes
changed_comparison_df.head()  # You can view the first few rows to inspect the changes


,camis,dba,boro,street,bbl,original_zipcode,new_zipcode
25,50092870,PAPA THAI,Manhattan,FREDERICK DOUGLASS BOULEVARD,1.0,NaN,NaN
31,50067032,DEATH AVENUE: TAP ROOM,Manhattan,10TH AVE,NaN,10001.0,NaN
52,41584120,7 SPICES,Bronx,E GUNHILL RD,2.0,NaN,NaN
64,41683279,INTREPID SEA-AIR-SPACE MUSEUM,Manhattan,INTREPID SQ,1.0,NaN,NaN
70,41432668,APPLEBEE'S,Bronx,3 AVENUE,2.0,NaN,NaN


In [33]:
# Create an empty DataFrame for comparison
comparison_df = pd.DataFrame(columns=['camis', 'dba', 'boro', 'street', 'bbl', 'original_zipcode', 'new_zipcode'])

# Populate the comparison DataFrame with relevant columns from the original DataFrame 'inspection_df'
comparison_df['camis'] = inspection_df['camis']
comparison_df['dba'] = inspection_df['dba']
comparison_df['boro'] = inspection_df['boro']
comparison_df['street'] = inspection_df['street']
comparison_df['bbl'] = inspection_df['bbl']
comparison_df['original_zipcode'] = inspection_df['zipcode']

# Create a copy of the original DataFrame
temp_df = inspection_df.copy()

# Step 1: Define a function to get the mode for zipcodes
def get_mode(x):
    modes = x.mode()
    if not modes.empty:
        return modes.iloc[0]
    else:
        return x

# Step 2: Apply the function only to rows where 'zipcode' is NaN
# Create a mask for rows where 'zipcode' is NaN
mask = temp_df['bbl'].notna()

# Apply the mode-filling operation only to these rows
temp_df.loc[mask, 'bbl'] = temp_df[mask].groupby(['street', 'bbl'])['zipcode'].transform(lambda x: x.fillna(get_mode(x)))

# Update the comparison DataFrame
comparison_df['new_zipcode'] = temp_df['zipcode']

# Filter out rows where the original and new zipcodes are the same
changed_comparison_df = comparison_df[comparison_df['original_zipcode'] != comparison_df['new_zipcode']]

# Display the DataFrame with changed zipcodes
changed_comparison_df.head()  # You can view the first few rows to inspect the changes



,camis,dba,boro,street,bbl,original_zipcode,new_zipcode
25,50092870,PAPA THAI,Manhattan,FREDERICK DOUGLASS BOULEVARD,1.0,NaN,NaN
52,41584120,7 SPICES,Bronx,E GUNHILL RD,2.0,NaN,NaN
64,41683279,INTREPID SEA-AIR-SPACE MUSEUM,Manhattan,INTREPID SQ,1.0,NaN,NaN
70,41432668,APPLEBEE'S,Bronx,3 AVENUE,2.0,NaN,NaN
85,41543970,PAUSE CAFE,Manhattan,CLINTON SREET,1.0,NaN,NaN


In [37]:
changed_comparison_df[changed_comparison_df.new_zipcode.notna()]

,camis,dba,boro,street,bbl,original_zipcode,new_zipcode


In [1]:
# inspection_df = inspection_df.dropna(subset=['zipcode'])
# Group by 'bbl' and fill missing 'zipcode' values with the mode of each group
inspection_df['zipcode'] = inspection_df.groupby(['street','bbl'])['zipcode'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))

: 

In [27]:
null_counts_by_column = inspection_df.isnull().sum()
null_counts_by_column[null_counts_by_column > 0]

building              237
zipcode               357
phone                   6
latitude              138
longitude             138
community_board      2591
council_district     2595
census_tract         2595
bin                  3240
bbl                   344
nta                  2591
grade               73661
grade_date          76774
dtype: int64

In [28]:
inspection_df = inspection_df.dropna(subset=['latitude'])


In [ ]:
columns_to_fill = ['community_board', 'council_district', 'census_tract', 'bin', 'bbl', 'nta']

for col in columns_to_fill:
    # Group by 'street', 'zipcode', and 'boro', and fill missing values with the mode of each group
    inspection_df[col] = inspection_df.groupby(['street', 'zipcode', 'boro'])[col].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))


: 

: 

In [ ]:
inspection_df['building'] = inspection_df['building'].fillna("0")
inspection_df['phone'] = inspection_df['phone'].fillna("0")

In [ ]:
# Group by the specified columns and fill missing 'bin' values with the mode of each group
inspection_df['bin'] = inspection_df.groupby(['community_board', 'council_district', 'census_tract', 'bbl', 'nta'])['bin'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))
# Group by 'bbl' and fill missing 'bin' values with the mode of each group
inspection_df['bin'] = inspection_df.groupby('bbl')['bin'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))


In [ ]:
council_district_isna = inspection_df[inspection_df['council_district'].isna()]
council_district_isna 

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description,grade,grade_date
31,50067032,DEATH AVENUE: TAP ROOM,Manhattan,None,10TH AVE,10001.0,2126958080,American,2022-06-14T00:00:00.000,Violations were cited in the following area(s).,Not Critical,11.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10H,Proper sanitization not provided for utensil w...,A,2022-06-14T00:00:00.000
86,50112968,DELICE MACARONS,Manhattan,321 1/2,AMSTERDAM AVENUE,10023.0,6469639913,Bakery Products/Desserts,2022-08-09T00:00:00.000,Violations were cited in the following area(s).,Critical,25.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.780645,-73.980130,NaN,NaN,NaN,NaN,NaN,NaN,02B,Hot TCS food item not held at or above 140 °F.,NaN,NaN
108,50063280,ENZO'S OF ARTHUR AVE,Bronx,None,ARTHUR AVE,10458.0,7187334455,Italian,2021-09-12T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:08.000,Inter-Agency Task Force / Initial Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,No violations were recorded,NaN,NaN
199,50086391,FIX-U-PLATE,Brooklyn,1139/1141,CLARKSON AVE,11212.0,9292343888,Caribbean,2023-04-05T00:00:00.000,Violations were cited in the following area(s).,Not Critical,44.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08A,Establishment is not free of harborage or cond...,NaN,NaN
279,41543919,CROQUE MADAME,Queens,None,JFK INTERNATIONAL AIRPORT,11430.0,9292414616,American,2017-12-05T00:00:00.000,Violations were cited in the following area(s).,Not Critical,2.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.648313,-73.788281,NaN,NaN,NaN,NaN,NaN,NaN,09C,Food contact surface not properly maintained.,A,2017-12-05T00:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204904,40649472,PIER 60,Manhattan,None,AND WEST SIDE HIGHWAY,10011.0,2123366060,American,2021-07-30T00:00:00.000,Violations were cited in the following area(s).,Not Critical,2.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10F,Non-food contact surface improperly constructe...,A,2021-07-30T00:00:00.000
204915,50113390,BREAKFAST BY SALTS CURE,Manhattan,27 1/2,MORTON STREET,10014.0,4432266388,American,2023-02-21T00:00:00.000,Violations were cited in the following area(s).,Critical,43.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.731022,-74.004572,NaN,NaN,NaN,NaN,NaN,NaN,06D,"Food contact surface not properly washed, rins...",NaN,NaN
205171,41414797,(PUBLIC FARE) 81st street and central park wes...,Manhattan,None,CENTRAL PARK WEST,10024.0,6467475354,American,2019-05-21T00:00:00.000,Violations were cited in the following area(s).,Critical,9.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06D,"Food contact surface not properly washed, rins...",A,2019-05-21T00:00:00.000
205678,41521007,"PRIME TAVERN, DELTA TERMINAL",Queens,None,US AIRWAYS TERMINAL,11371.0,7186566210,American,2017-09-14T00:00:00.000,Violations were cited in the following area(s).,Critical,9.0,2023-12-01T06:00:08.000,Cycle Inspection / Re-inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04H,"Raw, cooked or prepared food is adulterated, c...",A,2017-09-14T00:00:00.000




we should infer score = 0 and violation description = No Violations and code = No_violation. Then we can simply drop the 7  rows with action = Violations were cited as a possible error in the data. 

In [ ]:
# Step 1: Group by 'camis' and 'inspection_date' and check if there are different 'inspection_type' values within each group
grouped = inspection_df.groupby(['camis', 'inspection_date'])
diff_inspection_type = grouped['inspection_type'].nunique() > 1

# Step 2: Filter the DataFrame to include only those groups
filtered_df = inspection_df[inspection_df.set_index(['camis', 'inspection_date']).index.isin(diff_inspection_type[diff_inspection_type].index)].reset_index(drop=True)

# Now, 'filtered_df' contains only the groups where 'inspection_type' varies within the same 'camis' and 'inspection_date'
filtered_df.sort_values(by='camis').head(3)

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description,grade,grade_date
4587,30112340,WENDY'S,Brooklyn,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2022-07-13T00:00:00.000,Violations were cited in the following area(s).,Critical,11.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.662652,-73.962081,309.0,40.0,32700.0,3029737.0,3.011970e+09,BK60,02G,Cold TCS food item held above 41 °F; smoked or...,A,2022-07-13T00:00:00.000
3105,30112340,WENDY'S,Brooklyn,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2022-07-13T00:00:00.000,Violations were cited in the following area(s).,Not Critical,11.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.662652,-73.962081,309.0,40.0,32700.0,3029737.0,3.011970e+09,BK60,10F,Non-food contact surface or equipment made of ...,A,2022-07-13T00:00:00.000
1953,30112340,WENDY'S,Brooklyn,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2022-07-13T00:00:00.000,Violations were cited in the following area(s).,Not Critical,NaN,2023-12-01T06:00:08.000,Administrative Miscellaneous / Initial Inspection,40.662652,-73.962081,309.0,40.0,32700.0,3029737.0,3.011970e+09,BK60,20-06,Current letter grade or Grade Pending card not...,NaN,NaN


In [ ]:
# Step 1: Determine the mode 'inspection_type' for each group
mode_inspection_type = inspection_df.groupby(['camis', 'inspection_date'])['inspection_type'].agg(lambda x: x.mode()[0]).reset_index()
mode_inspection_type.rename(columns={'inspection_type': 'mode_inspection_type'}, inplace=True)

# Step 2: Merge this mode information back into the original DataFrame
merged_df = inspection_df.merge(mode_inspection_type, on=['camis', 'inspection_date'], how='left')

# Step 3: Filter out rows where 'inspection_type' is the mode
non_mode_df = merged_df[merged_df['inspection_type'] != merged_df['mode_inspection_type']]

# Filter out rows where 'inspection_type' starts with "Cycle Inspection"
non_mode_df = non_mode_df[~non_mode_df['inspection_type'].str.startswith("Cycle Inspection")]

# Step 4: Get unique 'inspection_type' values that are not the mode
unique_non_mode_types = non_mode_df['inspection_type'].unique().tolist()

# Now, 'unique_non_mode_types' contains the unique list of 'inspection_type' values that are not the mode in their group
unique_non_mode_types

['Administrative Miscellaneous / Re-inspection',
 'Administrative Miscellaneous / Initial Inspection',
 'Administrative Miscellaneous / Compliance Inspection',
 'Administrative Miscellaneous / Reopening Inspection',
 'Inter-Agency Task Force / Initial Inspection']

In [ ]:
# Step 4: Get unique 'inspection_type' values that are not the mode
unique_non_mode_violation_description = non_mode_df['violation_description'].unique().tolist()

# Now, 'unique_non_mode_types' contains the unique list of 'inspection_type' values that are not the mode in their group
unique_non_mode_violation_description

[nan,
 'MISBRANDED AND LABELING',
 'Current letter grade sign not posted.',
 'Failure to post or conspicuously post healthy eating information',
 'Failure to display required signage about plastic straw availability.',
 '“Choking first aid” poster not posted. “Alcohol and Pregnancy” warning sign not posted. Resuscitation equipment: exhaled air resuscitation masks (adult & pediatric), latex gloves, sign not posted.',
 'Nuisance created or allowed to exist. Facility not free from unsafe, hazardous, offensive or annoying condition.',
 'Failure to display required signage about plastic straw availability',
 'Current letter grade or "Grade Pending" card not posted.',
 'Providing single-use, non-compostable plastic straws to customers without customer request (including providing such straws at a self-serve station)',
 'Current letter grade or Grade Pending card not posted',
 'Permit not conspicuously displayed.',
 'Nuisance created or allowed to exist.  Facility not free from unsafe, hazard

In [ ]:
# Step 4: Get unique 'inspection_type' values that are not the mode
unique_non_mode_violation_description = non_mode_df['violation_code'].unique().tolist()

# Now, 'unique_non_mode_types' contains the unique list of 'inspection_type' values that are not the mode in their group
unique_non_mode_violation_description

[nan,
 '22F',
 '20F',
 '20-08',
 '19-10',
 '20-04',
 '28-01',
 '20-06',
 '19-06',
 '18-08',
 '22A',
 '20A',
 '19-07',
 '18-11',
 '22C',
 '20D',
 '18G',
 '18D',
 '22G',
 '20E',
 '20-01',
 '19-05',
 '19-11',
 '28-03',
 '19-03',
 '18-01',
 '20-05',
 '22E',
 '28-04',
 '18C',
 '19-04',
 '20-07',
 '18F',
 '04J',
 '18B',
 '04A',
 '18-14',
 '18-02',
 '18-13',
 '19-01',
 '19-08']

In [ ]:
# Filter out rows where 'inspection_type' starts with "Cycle Inspection"
task_force_df_ = non_mode_df[non_mode_df['inspection_type'].str.startswith("Inter-Agency")]

# Step 4: Get unique 'inspection_type' values that are not the mode
unique_non_mode_violation_description = task_force_df_['violation_description'].unique().tolist()

# Now, 'unique_non_mode_types' contains the unique list of 'inspection_type' values that are not the mode in their group
unique_non_mode_violation_description

['Appropriately scaled metal stem-type thermometer or thermocouple not provided or used to evaluate temperatures of potentially hazardous foods during cooking, cooling, reheating and holding.',
 'Food Protection Certificate (FPC) not held by manager or supervisor of food operations.']

In [ ]:
task_force_df_

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description,grade,grade_date,mode_inspection_type
35230,41646811,EL PATRON NIGHTCLUB CABARET-RESTAURANT,Bronx,1465,JEROME AVENUE,10452.0,6462280275,American,2016-11-05T00:00:00.000,Violations were cited in the following area(s).,Critical,8.0,2023-12-01T06:00:08.000,Inter-Agency Task Force / Initial Inspection,40.841056,-73.917106,204.0,16.0,21900.0,2008317.0,2.028580e+09,BX26,04J,Appropriately scaled metal stem-type thermomet...,NaN,NaN,Administrative Miscellaneous / Initial Inspection
63888,50081955,STAND UP CAFE,Brooklyn,7215,18 AVENUE,11204.0,7182328866,Chinese,2022-09-16T00:00:00.000,Violations were cited in the following area(s).,Critical,10.0,2023-12-01T06:00:08.000,Inter-Agency Task Force / Initial Inspection,40.614978,-73.994360,311.0,43.0,26600.0,3158517.0,3.061940e+09,BK28,04A,Food Protection Certificate (FPC) not held by ...,NaN,NaN,Administrative Miscellaneous / Initial Inspection


In [ ]:
inspection_vscore =  inspection_df[inspection_df.score.isna()]
inspection_vcode =  inspection_df[inspection_df.violation_code.isna()]
len(inspection_vcode), len(inspection_vscore)

(864, 6121)

In [ ]:
inspection_df[inspection_df['inspection_type'] == 'Administrative Miscellaneous / Initial Inspection']

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description,grade,grade_date
15,50087361,PROOF COFFEE ROASTERS,Manhattan,2286,7 AVENUE,10030.0,3142300768,Coffee/Tea,2023-06-27T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,NaN,2023-12-01T06:00:08.000,Administrative Miscellaneous / Initial Inspection,40.814888,-73.944014,110.0,9.0,22800.0,1058842.0,1.019400e+09,MN03,NaN,NaN,NaN,NaN
23,50039250,LIN'S GARDEN,Bronx,3986,WHITE PLAINS ROAD,10466.0,7185190135,Chinese,2023-04-27T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,NaN,2023-12-01T06:00:08.000,Administrative Miscellaneous / Initial Inspection,40.888148,-73.860254,212.0,12.0,40600.0,2063440.0,2.048390e+09,BX44,NaN,NaN,NaN,NaN
30,50050386,PANDA EXPRESS,Manhattan,835,3 AVENUE,10022.0,2123552821,Chinese,2022-07-20T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,NaN,2023-12-01T06:00:09.000,Administrative Miscellaneous / Initial Inspection,40.756303,-73.970402,106.0,4.0,9800.0,1081187.0,1.013240e+09,MN19,NaN,NaN,NaN,NaN
55,50092607,JALLOH FAMILY RESTAURANT,Bronx,3396,3 AVENUE,10456.0,9176595169,African,2023-05-16T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,NaN,2023-12-01T06:00:08.000,Administrative Miscellaneous / Initial Inspection,40.827899,-73.907168,203.0,17.0,18500.0,2004198.0,2.026080e+09,BX35,NaN,NaN,NaN,NaN
72,50018434,SEA WOLF,Brooklyn,19,WYCKOFF AVENUE,11237.0,7183883265,American,2022-07-13T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,NaN,2023-12-01T06:00:09.000,Administrative Miscellaneous / Initial Inspection,40.706642,-73.922966,304.0,34.0,44700.0,3072408.0,3.031770e+09,BK77,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206948,50127672,TBAR,Manhattan,116,EAST 60 STREET,10022.0,2127720404,American,2022-09-29T00:00:00.000,Violations were cited in the following area(s).,Not Critical,NaN,2023-12-01T06:00:08.000,Administrative Miscellaneous / Initial Inspection,40.763189,-73.968540,108.0,4.0,11402.0,1041906.0,1.013940e+09,MN40,20-04,“Choking first aid” poster not posted. “Alcoho...,NaN,NaN
206949,50090163,PLACE TO BEACH CANTINA,Brooklyn,1301,BOARDWALK WEST,11224.0,7183735862,American,2019-07-13T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,NaN,2023-12-01T06:00:09.000,Administrative Miscellaneous / Initial Inspection,40.573047,-73.980865,313.0,47.0,35200.0,3189663.0,3.070740e+09,BK21,NaN,NaN,NaN,NaN
206989,50136766,Balila Labanese Restaurant,Manhattan,1640,3 AVENUE,10128.0,2014107189,Middle Eastern,2023-06-29T00:00:00.000,Violations were cited in the following area(s).,Not Critical,NaN,2023-12-01T06:00:08.000,Administrative Miscellaneous / Initial Inspection,40.782492,-73.951310,108.0,5.0,15801.0,1078536.0,1.015200e+09,MN40,20-04,“Choking first aid” poster not posted. “Alcoho...,NaN,NaN
207199,41630950,CITY LINE PIZZA & PASTA,Brooklyn,1224,LIBERTY AVENUE,11208.0,7182774992,Pizza,2021-08-25T00:00:00.000,Violations were cited in the following area(s).,Not Critical,NaN,2023-12-01T06:00:08.000,Administrative Miscellaneous / Initial Inspection,40.679015,-73.863896,305.0,37.0,118800.0,3094524.0,3.042060e+09,BK83,22C,"Bulb not shielded or shatterproof, in areas wh...",NaN,NaN


In [ ]:
vcode_date = inspection_vcode.sort_values(by=['inspection_date'], ascending=False)
vcode_date.sort_values(by=['camis'], ascending=False)

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description,grade,grade_date
197017,50144058,RESPITE CENTERS (STEWART HOTEL),Manhattan,371,7 AVENUE,10001.0,9178480367,American,2023-11-22T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:08.000,Pre-permit (Non-operational) / Initial Inspection,40.748799,-73.992204,105.0,3.0,10100.0,1015174.0,1.008060e+09,MN17,NaN,NaN,N,NaN
84323,50143779,ROSE VALLEY CAKES BY KAVITA & KAMLA,Queens,25501B,UNION TPKE,11004.0,9174953351,Bakery Products/Desserts,2023-11-13T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:08.000,Pre-permit (Non-operational) / Initial Inspection,40.745066,-73.715683,413.0,23.0,155102.0,4440956.0,4.085130e+09,QN44,NaN,NaN,N,NaN
136278,50143605,NEPALI MOMO CAFE,Queens,7418,37TH RD,11372.0,3477384199,Asian/Asian Fusion,2023-11-13T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:08.000,Pre-permit (Non-operational) / Initial Inspection,40.747252,-73.891246,403.0,25.0,28900.0,4029839.0,4.012850e+09,QN28,NaN,NaN,N,NaN
47333,50143038,ZEN KITCHEN,Brooklyn,1736,SHORE PARKWAY,11214.0,7187999134,"Juice, Smoothies, Fruit Salads",2023-10-18T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:08.000,Pre-permit (Non-operational) / Initial Inspection,40.593522,-73.996817,311.0,43.0,30400.0,3170082.0,3.064910e+09,BK29,NaN,NaN,N,NaN
50885,50141803,L'INDUSTRIE PIZZERIA,Manhattan,104,CHRISTOPHER STREET,10014.0,2122560648,Pizza,2023-10-26T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:09.000,Pre-permit (Operational) / Initial Inspection,40.733275,-74.004925,102.0,3.0,7300.0,1010039.0,1.005880e+09,MN23,NaN,NaN,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106964,40376515,AMERICAN MUSEUM OF NATURAL HISTORY FOOD COURT,Manhattan,None,W 79 STREET,10024.0,2127695370,American,2023-06-21T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,2023-06-21T00:00:00.000
145390,40367339,COCKTAIL ROOM (NYAC),Manhattan,180,CENTRAL PARK SOUTH,10019.0,2127677000,American,2022-02-16T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.766718,-73.978737,105.0,4.0,13700.0,1023750.0,1.010110e+09,MN17,NaN,NaN,A,2022-02-16T00:00:00.000
117370,40365942,HOP KEE RESTAURANT,Manhattan,21,MOTT STREET,10013.0,2129648365,Chinese,2022-09-23T00:00:00.000,Establishment re-opened by DOHMH.,Not Applicable,0.0,2023-12-01T06:00:08.000,Cycle Inspection / Reopening Inspection,40.714364,-73.998784,103.0,1.0,2900.0,1001821.0,1.001640e+09,MN27,NaN,NaN,C,2022-09-23T00:00:00.000
53835,40365784,GARGIULO'S RESTAURANT,Brooklyn,2911,WEST 15 STREET,11224.0,7182664891,Italian,2023-03-16T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,0.0,2023-12-01T06:00:09.000,Cycle Inspection / Initial Inspection,40.576555,-73.982584,313.0,47.0,34800.0,3339622.0,3.070640e+09,BK21,NaN,NaN,NaN,NaN


In [ ]:
# Group by 'camis' and take the first non-null 'community_board' value
community_board_mapping = inspection_df.groupby('camis')['community_board'].first()

# Now, fill missing community_board values using this mapping
inspection_df['community_board'] = inspection_df['community_board'].fillna(inspection_df['camis'].map(community_board_mapping))

# Check the count of remaining missing values in 'community_board'
missing_after = inspection_df['community_board'].isna().sum()
print(f"Remaining missing community_board numbers: {missing_after}")


Remaining missing community_board numbers: 3153


In [ ]:
inspection_df[inspection_df['camis'] == 41360872].sort_values(by='inspection_date')

,camis,dba,boro,building,street,zipcode,phone,cuisine_description,inspection_date,action,critical_flag,score,record_date,inspection_type,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta,violation_code,violation_description,grade,grade_date
125478,41360872,DUNKIN,Staten Island,2945,VETERANS ROAD WEST,10309.0,7189843717,Donuts,2022-03-30T00:00:00.000,Violations were cited in the following area(s).,Not Critical,3.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.527753,-74.231871,503.0,51.0,22600.0,5150130.0,5.075110e+09,SI11,10F,Non-food contact surface improperly constructe...,A,2022-03-30T00:00:00.000
106069,41360872,DUNKIN,Staten Island,2945,VETERANS ROAD WEST,10309.0,7189843717,Donuts,2023-04-05T00:00:00.000,Violations were cited in the following area(s).,Not Critical,4.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.527753,-74.231871,503.0,51.0,22600.0,5150130.0,5.075110e+09,SI11,10F,Non-food contact surface or equipment made of ...,A,2023-04-05T00:00:00.000
197602,41360872,DUNKIN,Staten Island,2945,VETERANS ROAD WEST,10309.0,7189843717,Donuts,2023-04-05T00:00:00.000,Violations were cited in the following area(s).,Not Critical,4.0,2023-12-01T06:00:08.000,Cycle Inspection / Initial Inspection,40.527753,-74.231871,503.0,51.0,22600.0,5150130.0,5.075110e+09,SI11,10E,Accurate thermometer not provided or properly ...,A,2023-04-05T00:00:00.000
205818,41360872,DUNKIN,Staten Island,2945,VETERANS ROAD WEST,10309.0,7189843717,Donuts,2023-04-05T00:00:00.000,No violations were recorded at the time of thi...,Not Applicable,NaN,2023-12-01T06:00:09.000,Administrative Miscellaneous / Initial Inspection,40.527753,-74.231871,503.0,51.0,22600.0,5150130.0,5.075110e+09,SI11,NaN,NaN,NaN,NaN
